<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [34]:
#!/usr/bin/env python
# coding: utf-8

# <h1>Table of Contents<span class="tocSkip"></span></h1>
# <div class="toc"><ul class="toc-item"></ul></div>
# Это мы пробуем с Qwen3

# In[1]:

from typing import List, Callable, Dict, Tuple, Optional
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline
import pandas as pd
from pathlib import Path

    
import time as tm
t1_itogo = tm.time() # длЯ расчета времени исполнения всей тетрадки

pd.set_option("display.max_rows", None)  #убрали ограничение на число строк при отображении датафрейма
try:
    from IPython.display import display
    HAVE_DISPLAY = True
except Exception:
    HAVE_DISPLAY = False
    
    
import time as tm
t1_itogo = tm.time() # длЯ расчета времени исполнения всей тетрадки



USE_FILE_PROMPTS = False
PROMPT_FILE = "prompt.txt"
PROMPTS = (
    [Path(PROMPT_FILE).read_text(encoding="utf-8").strip()] if USE_FILE_PROMPTS else [
        "Write me a code for sorting an array in Python",
    ]
)

# --- Qwen3 chat template (для instruct-моделей)
USE_CHAT_TEMPLATE = True   # <<< добавлено: применять чат-шаблон, если он есть у токенизатора
QWEN_THINKING = False      # <<< добавлено: отключаем "thinking" для детерминированного greedy


MODS = [
    ("original", lambda p: p),
   # ("typo first e", lambda p: p.replace("e", "3", 1)),
  #  ("add salutation", lambda p: "Dear user, " + p),
  #  ("префикс ======", lambda p: "=" * 10 + p),
    ("префикс вопрос", lambda p: "I have a question. " + p),
  #  ("суффикс 10 лет?", lambda p: p + " in the next decade?"),
 
]

MODEL_NAME = "Qwen/Qwen3-0.6B"  # <<< изменено: был "gpt2"
STEPS = 50

# --------------------------------------------------
# 1.  Loading
# --------------------------------------------------

def setup_model(model_name: str = "gpt2", device: Optional[str] = None, dtype: Optional[torch.dtype] = None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    tok = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    if tok.pad_token_id is None:
        tok.pad_token = tok.eos_token
    # <<< изменено: позволяем HF выбрать тип автоматически (BF16/FP16 на GPU, FP32 на CPU)
    mk = {"torch_dtype": dtype or "auto"}
    mdl = AutoModelForCausalLM.from_pretrained(model_name, **mk).to(device)
    if device == "cuda":
        torch.backends.cuda.matmul.allow_tf32 = True
    return mdl, tok, device

# --------------------------------------------------
# 2.  Batch preparation
# --------------------------------------------------

def prepare_batch(
    prompts: List[str],
    mods: List[Tuple[str, Callable[[str], str]]],
    tokenizer,
    device: str,
):
    texts, meta = [], []  # meta: (prompt_idx, mod_name)
    for i, p in enumerate(prompts):
        for m_name, m_fn in mods:
            meta.append((i, m_name))
            txt = m_fn(p)
            # <<< добавлено: применяем чат-шаблон Qwen3 при наличии
            if USE_CHAT_TEMPLATE and hasattr(tokenizer, "apply_chat_template") and getattr(tokenizer, "chat_template", None):
                messages = [{"role": "user", "content": txt}]
                txt = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True,
                    enable_thinking=QWEN_THINKING,
                )
            texts.append(txt)
    enc = tokenizer(texts, padding=True, return_tensors="pt")
    enc = {k: v.to(device) for k, v in enc.items()}
    lens = enc["attention_mask"].sum(dim=1)
    return enc, lens, meta

# --------------------------------------------------
# 3.  Generation wrapper
# --------------------------------------------------

def run_generation(model, enc, steps: int, pad_id: int):
    cfg = GenerationConfig(max_new_tokens=steps, do_sample=False, num_beams=1, pad_token_id=pad_id, use_cache=True)
    model.eval()
    with torch.inference_mode():
        out = model.generate(**enc, generation_config=cfg)
    return out

# --------------------------------------------------
# 4.  Post‑processing
# --------------------------------------------------

def decode_results(
    out, lens, meta, tokenizer, steps: int, decode_tokens: bool
) -> Dict[int, Dict[str, List]]:
    res: Dict[int, Dict[str, List]] = {}
    for idx, (p_idx, m_name) in enumerate(meta):
        start = lens[idx].item()
        ids = out[idx][start : start + steps].tolist()
        tokens = [tokenizer.decode([i], skip_special_tokens=True) for i in ids] if decode_tokens else ids
        res.setdefault(p_idx, {})[m_name] = tokens
    return res

# --------------------------------------------------
# 5.  Pretty printing
# --------------------------------------------------

def show_tables(res, prompts, trunc: int, return_dfs: bool, print_text: bool = True, print_table: bool = False): 
    #если print_text: bool = True, то делаем печать помимо DataFrame
    dfs = {}
    for i, p in enumerate(prompts):
        df = pd.DataFrame(res[i])
        df.index = [f"step {j+1}" for j in range(len(df))]
        hdr = f"=== PROMPT {i+1}: {p[:trunc]}{'…' if len(p) > trunc else ''} ==="
        


        # Таблица top‑1 токенов
        if print_table:  # <<< добавлено: можно отключить печать таблицы
            print("\n" + hdr)
            if HAVE_DISPLAY:
                display(df)
            else:
                print(df.to_string())   
   
        # Краткий текст‑продолжение из n токенов
        
        if print_text:
            print("\n" + hdr)
            print("\n--- concatenated top‑1 continuation (n = steps) ---")
            for col in df.columns:
                joined = ''.join(res[i][col])  # токены могут содержать пробелы
                print(f"[{col}] {joined}")
                print()

        if return_dfs:
            dfs[i] = df
    return dfs if return_dfs else None



# In[2]:

# --------------------------------------------------
# 6.  Public API
# --------------------------------------------------

def greedy_top1(
    prompts: List[str],
    mods: List[Tuple[str, Callable[[str], str]]],
    model, tokenizer,
    steps: int = 40,
    device: Optional[str] = None,
    decode_tokens: bool = True,
    trunc_print: int = 120,
    return_dataframes: bool = True,
    print_text: bool = True,      # <<< добавлено: пробрасываем флаг печати текста
    print_table: bool = False,     # <<< добавлено: пробрасываем флаг печати таблицы
):
    device = device or model.device
    enc, lens, meta = prepare_batch(prompts, mods, tokenizer, device)
    out = run_generation(model, enc, steps, tokenizer.pad_token_id)
    res = decode_results(out, lens, meta, tokenizer, steps, decode_tokens)
    dfs = show_tables(res, prompts, trunc_print, return_dataframes)
    return (res, dfs) if return_dataframes else res

# In[3]:

# --------------------------------------------------
# 7.  Script example
# --------------------------------------------------

mdl, tok, dev = setup_model(MODEL_NAME)
_ = greedy_top1(PROMPTS, MODS, mdl, tok, steps=STEPS, device=dev)


# In[4]:

t2_itogo = tm.time()
print('вся тетрадка заняла', round(t2_itogo - t1_itogo)//60,'минут', round(t2_itogo - t1_itogo)%60,'секунд')



=== PROMPT 1: Write me a code for sorting an array in Python ===

--- concatenated top‑1 continuation (n = steps) ---
[original] 
<think>

</think>

Sure! Here's a simple Python code to sort an array:

```python
# Example usage
arr = [5, 2, 8, 0, 1]

# Sort the array in ascending order


[префикс вопрос] Sure! Here's a simple example of how to sort an array in Python:

```python
# Example array
my_array = [5, 2, 8, 0, 1, 4]

# Sort the array in ascending

вся тетрадка заняла 1 минут 22 секунд


In [33]:
#!/usr/bin/env python
# coding: utf-8

# <h1>Table of Contents<span class="tocSkip"></span></h1>
# <div class="toc"><ul class="toc-item"></ul></div>
# Это мы пробуем с gpt2

# In[1]:

from typing import List, Callable, Dict, Tuple, Optional
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline
import pandas as pd
from pathlib import Path

    
import time as tm
t1_itogo = tm.time() # длЯ расчета времени исполнения всей тетрадки

pd.set_option("display.max_rows", None)  #убрали ограничение на число строк при отображении датафрейма
try:
    from IPython.display import display
    HAVE_DISPLAY = True
except Exception:
    HAVE_DISPLAY = False
    
    
import time as tm
t1_itogo = tm.time() # длЯ расчета времени исполнения всей тетрадки



USE_FILE_PROMPTS = False
PROMPT_FILE = "prompt.txt"
PROMPTS = (
    [Path(PROMPT_FILE).read_text(encoding="utf-8").strip()] if USE_FILE_PROMPTS else [
        "Write me a code for sorting an array in Python",
    ]
)

# --- Qwen3 chat template (для instruct-моделей)
USE_CHAT_TEMPLATE = True   # <<< добавлено: применять чат-шаблон, если он есть у токенизатора
QWEN_THINKING = False      # <<< добавлено: отключаем "thinking" для детерминированного greedy


MODS = [
    ("original", lambda p: p),
   # ("typo first e", lambda p: p.replace("e", "3", 1)),
  #  ("add salutation", lambda p: "Dear user, " + p),
  #  ("префикс ======", lambda p: "=" * 10 + p),
    ("префикс вопрос", lambda p: "I have a question. " + p),
  #  ("суффикс 10 лет?", lambda p: p + " in the next decade?"),
 
]

MODEL_NAME = "gpt2"  # <<< изменено: был "gpt2"
STEPS = 10

# --------------------------------------------------
# 1.  Loading
# --------------------------------------------------

def setup_model(model_name: str = "gpt2", device: Optional[str] = None, dtype: Optional[torch.dtype] = None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    tok = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    if tok.pad_token_id is None:
        tok.pad_token = tok.eos_token
    # <<< изменено: позволяем HF выбрать тип автоматически (BF16/FP16 на GPU, FP32 на CPU)
    mk = {"torch_dtype": dtype or "auto"}
    mdl = AutoModelForCausalLM.from_pretrained(model_name, **mk).to(device)
    if device == "cuda":
        torch.backends.cuda.matmul.allow_tf32 = True
    return mdl, tok, device

# --------------------------------------------------
# 2.  Batch preparation
# --------------------------------------------------

def prepare_batch(
    prompts: List[str],
    mods: List[Tuple[str, Callable[[str], str]]],
    tokenizer,
    device: str,
):
    texts, meta = [], []  # meta: (prompt_idx, mod_name)
    for i, p in enumerate(prompts):
        for m_name, m_fn in mods:
            meta.append((i, m_name))
            txt = m_fn(p)
            # <<< добавлено: применяем чат-шаблон Qwen3 при наличии
            if USE_CHAT_TEMPLATE and hasattr(tokenizer, "apply_chat_template") and getattr(tokenizer, "chat_template", None):
                messages = [{"role": "user", "content": txt}]
                txt = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True,
                    enable_thinking=QWEN_THINKING,
                )
            texts.append(txt)
    enc = tokenizer(texts, padding=True, return_tensors="pt")
    enc = {k: v.to(device) for k, v in enc.items()}
    lens = enc["attention_mask"].sum(dim=1)
    return enc, lens, meta

# --------------------------------------------------
# 3.  Generation wrapper
# --------------------------------------------------

def run_generation(model, enc, steps: int, pad_id: int):
    cfg = GenerationConfig(max_new_tokens=steps, do_sample=False, num_beams=1, pad_token_id=pad_id, use_cache=True)
    model.eval()
    with torch.inference_mode():
        out = model.generate(**enc, generation_config=cfg)
    return out

# --------------------------------------------------
# 4.  Post‑processing
# --------------------------------------------------

def decode_results(
    out, lens, meta, tokenizer, steps: int, decode_tokens: bool
) -> Dict[int, Dict[str, List]]:
    res: Dict[int, Dict[str, List]] = {}
    for idx, (p_idx, m_name) in enumerate(meta):
        start = lens[idx].item()
        ids = out[idx][start : start + steps].tolist()
        tokens = [tokenizer.decode([i], skip_special_tokens=True) for i in ids] if decode_tokens else ids
        res.setdefault(p_idx, {})[m_name] = tokens
    return res

# --------------------------------------------------
# 5.  Pretty printing
# --------------------------------------------------

def show_tables(res, prompts, trunc: int, return_dfs: bool, print_text: bool = True, print_table: bool = False,
               model_name: Optional[str] = None, device: Optional[str] = None,
                steps_value: Optional[int] = None): 
    #если print_text: bool = True, то делаем печать помимо DataFrame
    dfs = {}
    # <<< добавлено: печать информации о модели один раз
    if model_name or device:
        print(f"=== MODEL: {model_name or 'unknown'} | device: {device or 'unknown'} ===")
    for i, p in enumerate(prompts):
        df = pd.DataFrame(res[i])
        df.index = [f"step {j+1}" for j in range(len(df))]
        hdr = f"=== PROMPT {i+1}: {p[:trunc]}{'…' if len(p) > trunc else ''} ==="
        


        # Таблица top‑1 токенов
        if print_table:  # <<< добавлено: можно отключить печать таблицы
            print("\n" + hdr)
            if HAVE_DISPLAY:
                display(df)
            else:
                print(df.to_string())   
   
        # Краткий текст‑продолжение из n токенов
        
        if print_text:
            print("\n" + hdr)
            print("\n--- concatenated top‑1 continuation (n = steps) ---")
            for col in df.columns:
                joined = ''.join(res[i][col])  # токены могут содержать пробелы
                print(f"[{col}] {joined}")
                print()

        if return_dfs:
            dfs[i] = df
    return dfs if return_dfs else None



# In[2]:

# --------------------------------------------------
# 6.  Public API
# --------------------------------------------------

def greedy_top1(
    prompts: List[str],
    mods: List[Tuple[str, Callable[[str], str]]],
    model, tokenizer,
    steps: int = 40,
    device: Optional[str] = None,
    decode_tokens: bool = True,
    trunc_print: int = 120,
    return_dataframes: bool = True,
    print_text: bool = True,      # <<< добавлено: пробрасываем флаг печати текста
    print_table: bool = False,     # <<< добавлено: пробрасываем флаг печати таблицы
):
    device = device or model.device
    enc, lens, meta = prepare_batch(prompts, mods, tokenizer, device)
    out = run_generation(model, enc, steps, tokenizer.pad_token_id)
    res = decode_results(out, lens, meta, tokenizer, steps, decode_tokens)
     # <<< добавлено: передаём сведения о модели/устройстве и число шагов
    model_name = getattr(model, 'name_or_path', None) or (MODEL_NAME if 'MODEL_NAME' in globals() else None)
    dfs = show_tables(res, prompts, trunc_print, return_dataframes)
    return (res, dfs) if return_dataframes else res

# In[3]:

# --------------------------------------------------
# 7.  Script example
# --------------------------------------------------

mdl, tok, dev = setup_model(MODEL_NAME)
_ = greedy_top1(PROMPTS, MODS, mdl, tok, steps=STEPS, device=dev)


# In[4]:

t2_itogo = tm.time()
print('вся тетрадка заняла', round(t2_itogo - t1_itogo)//60,'минут', round(t2_itogo - t1_itogo)%60,'секунд')



=== PROMPT 1: Write me a code for sorting an array in Python ===

--- concatenated top‑1 continuation (n = steps) ---
[original]  sorting an array in Python.

import random

[префикс вопрос] .

I have a question. Write me

вся тетрадка заняла 0 минут 4 секунд
